# Part II of intersection detection
In this part, intersection circles are determined, where the conditions are relexed to ensure more possible intersections are detected. Part I is in `IntersectionDetection.py`

In [ ]:
from shapely.geometry import MultiPoint, LineString
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import numpy as np
from tqdm import tqdm
from shapely import MultiLineString, LineString
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
font = {'family' : 'Arial', 'size'   : 9}
plt.rc('font', **font)
plt.rcParams['mathtext.fontset'] = 'cm'
from IPython.display import display, clear_output
import time
import os

data_path = os.path.abspath('..') + '/OutputData/DriverSpace/Intersection/' # Set your own path here

def rotateMatrix(a):
    return np.array([[np.cos(a), -np.sin(a)], [np.sin(a), np.cos(a)]])

## Determining the circle boundaries of each potential intersection

In [ ]:
intersection_list = []
for dx in tqdm(['d'+str(did+1) for did in range(10)]):
    data = pd.read_hdf(os.path.abspath('..') + '/InputData/pNEUMA/'+dx+'/data_20181029_'+dx+'_0930_1000.h5', key='data')
    roads = pd.read_hdf(data_path+'trajectories/'+dx+'_roads.h5', key='data')
    hotspots = roads[(roads['G_star']<np.percentile(roads['G_star'],15))|
                                    (roads['G_star']>np.percentile(roads['G_star'],85))][['x','y']]
    
    
    clustering = AgglomerativeClustering(n_clusters=None, 
                                         distance_threshold=25, 
                                         affinity='euclidean',
                                         linkage='average',
                                         compute_full_tree=True).fit(hotspots)
    hotspots['cluster'] = clustering.labels_

    intersections = []
    for cluster in hotspots['cluster'].unique():
        intersect_points = hotspots[hotspots['cluster']==cluster][['x','y']]
        if len(intersect_points)>3:
            multipoints = MultiPoint(intersect_points.values)
            convex_hull = multipoints.convex_hull
            mbr_points = list(zip(*convex_hull.minimum_rotated_rectangle.exterior.coords.xy))
            mbr_lengths = [LineString((mbr_points[i], mbr_points[i+1])).length for i in range(len(mbr_points) - 1)]
            radius_min = min(mbr_lengths)
            radius_max = max(mbr_lengths)
            if radius_min > 3.5:
                df = data[((data['x']-convex_hull.centroid.x)**2+(data['y']-convex_hull.centroid.y)**2)<=(radius_max)**2].reset_index()
                traffic = len(df['track_id'].unique())/((df['time'].max()-df['time'].min())/60/60)/np.pi/radius_max**2
                intersections.append([cluster, convex_hull.centroid.x, convex_hull.centroid.y, radius_max, traffic])
    intersections = pd.DataFrame(intersections, columns=['id','x','y','radius','traffic'])
    intersections['dx'] = dx
    intersection_list.append(intersections)

# Save intersections
intersection_list = pd.concat(intersection_list)
intersection_list = intersection_list.sort_values(by=['dx','id']).reset_index(drop=True)
intersection_list.to_csv(data_path + 'potential_intersections.csv', index=False)

## Manually select intersections of interest
Repeat from d1 to d10

In [ ]:
# For the first time
intersections = pd.read_csv(data_path + 'original_intersections.csv')
intersections['type'] = 'Intersection'
intersections['keep'] = True
intersections['signalized'] = 1 # 1: signalized, 0: unsignalized, -1: not checked

In [ ]:
# For areas after d1
intersections = pd.read_csv(data_path + 'intersections_all.csv')

### Mark types of potential interesections

In [ ]:
dx = 'd6'
roads = pd.read_hdf(data_path+'trajectories/'+dx+'_roads.h5', key='data')
trajectories = pd.read_hdf(data_path+'trajectories/simplified_trajectories_'+dx+'.h5', key='data')
int2inspect = intersections[intersections['dx']==dx].copy()
x0, y0 = roads[['x','y']].mean().values
roads[['x','y']] = (roads[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
trajectories[['x','y']] = (trajectories[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
int2inspect[['x','y']] = (int2inspect[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

check_id = -1

In [ ]:
# Repeat to check all hotspots
check_id += 1
intersection = int2inspect.iloc[check_id]
data_int = trajectories[(trajectories['x']-intersection['x'])**2+(trajectories['y']-intersection['y'])**2<intersection['radius']**2]

fig, ax = plt.subplots(figsize=(4,4))
lines = [line[['x','y']].values for _, line in data_int.groupby('track_id')[['x','y']]]
lc = mpl.collections.LineCollection(lines, color='k', linewidth=0.1, alpha=1)
ax.add_collection(lc)
ax.scatter(data_int.groupby('track_id')['x'].first(), data_int.groupby('track_id')['y'].first(), s=5, c='g')
ax.scatter(data_int.groupby('track_id')['x'].last(), data_int.groupby('track_id')['y'].last(), s=5, c='r')
ax.scatter(intersection['x'], intersection['y'], s=50, marker='x', c='b', zorder=10)
circle = plt.Circle((intersection['x'], intersection['y']), intersection['radius'], color='b', fill=False)
ax.add_artist(circle)
ax.set_title('id: '+str(intersection['id']))
ax.set_aspect('equal')

In [ ]:
dx

In [ ]:
T_junctions_list = {'d1':[2,11], 'd2':[4,14,18], 
                    'd3':[13,19,23,37],'d4':[6,21,31], 
                    'd5':[10], 'd6': [8], 
                    'd7':[], 'd8':[6], 
                    'd9':[], 'd10':[]}
T_junctions = T_junctions_list[dx]
# Difference between T-junctions and On/off ramps: T-junctions have >=2 roads crossing
On_off_ramps_list = {'d1':[8,12,18,20,25], 'd2':[1,10,11,15], 
                     'd3':[2,3,5,8,11,14,20,21,22,25],'d4':[10,12], 
                     'd5':[5,6,7,8,9,11], 'd6': [4,14,18,21,26], 
                     'd7':[3,7,12,14], 'd8':[1,3,12,13,15,22,24,28,34,35], 
                     'd9':[1,12,15,39,40], 'd10':[2,6,8,15]}
On_off_ramps = On_off_ramps_list[dx]

Neither_list = {'d1':[], 'd2':[0,2,9,22], 
                'd3':[12,18,26,28,29],'d4':[1,5,13,14,33,40], 
                'd5':[0,2,4,14,15,16,19,20,21], 'd6': [17,20,23,30,36], 
                'd7':[0,1,2,8,26,30,33], 'd8':[5,8,10,11,25,37], 
                'd9':[2,8,19,23,41], 'd10':[0,1,3,7,14,16]}
Neither = Neither_list[dx]

intersections.loc[(intersections['dx']==dx)&(intersections['id'].isin(T_junctions)), 'type'] = 'T-junction'
intersections.loc[(intersections['dx']==dx)&(intersections['id'].isin(On_off_ramps)), 'type'] = 'On/off ramp'
intersections.loc[(intersections['dx']==dx)&(intersections['id'].isin(Neither)), 'type'] = 'Straight/curve road or video edge'

intersections.loc[(intersections['dx']==dx)&(~intersections['type'].isin(['Intersection','T-junction'])), 'keep'] = False
intersections.loc[(intersections['dx']==dx)&(~intersections['type'].isin(['Intersection','T-junction'])), 'signalized'] = -1

### Merge circles (if necessary)

In [ ]:
int2inspect = intersections[(intersections['dx']==dx)&(intersections['keep'])].copy()
int2inspect[['x','y']] = (int2inspect[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

plt.scatter(roads['x'], roads['y'], s=0.1, c='grey')
plt.scatter(int2inspect['x'], int2inspect['y'], s=5, c='b')
for i in range(len(int2inspect)):
    circle = plt.Circle((int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']), int2inspect.iloc[i]['radius'], color='b', fill=False)
    plt.gcf().gca().add_artist(circle)
    plt.text(int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']+int2inspect.iloc[i]['radius'], 
             str(int2inspect.iloc[i]['id'])+', r='+str(int(int2inspect.iloc[i]['radius'])), 
             fontsize=11, color='b')
plt.axis('equal')
plt.show()
print(int2inspect['id'].sort_values().values)

In [ ]:
merge_points_i_list = [[], [], [16,19,], [18], [], 
                       [], [], [], [], []]
merge_points_i = merge_points_i_list[int(dx[1:])-1]

merge_points_j_list = [[], [], [37,13,], [6], [],
                       [], [], [], [], []]
merge_points_j = merge_points_j_list[int(dx[1:])-1]

for i,j in zip(merge_points_i, merge_points_j):
    x_i, y_i, radius_i = intersections.loc[(intersections['dx']==dx)&(intersections['id']==i), ['x','y','radius']].values[0]
    x_j, y_j, radius_j = intersections.loc[(intersections['dx']==dx)&(intersections['id']==j), ['x','y','radius']].values[0]
    d = np.sqrt((x_i-x_j)**2+(y_i-y_j)**2)
    merged_radius = (radius_i + radius_j + d)/2
    merged_x = x_i + (x_j-x_i)*(merged_radius-radius_i)/d
    merged_y = y_i + (y_j-y_i)*(merged_radius-radius_i)/d
    intersections.loc[(intersections['dx']==dx)&(intersections['id']==i), ['x','y','radius']] = [merged_x, merged_y, merged_radius]
    intersections.loc[(intersections['dx']==dx)&(intersections['id']==j), 'keep'] = False
    intersections.loc[(intersections['dx']==dx)&(intersections['id']==j), 'note'] = 'Merged with '+str(i)

### Adjust radii
We don't move centers as they are harder to extimate how far should be moved

In [ ]:
int2inspect = intersections[(intersections['dx']==dx)&(intersections['keep'])].copy()
int2inspect[['x','y']] = (int2inspect[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

plt.scatter(roads['x'], roads['y'], s=0.1, c='grey')
plt.scatter(int2inspect['x'], int2inspect['y'], s=5, c='b')
for i in range(len(int2inspect)):
    circle = plt.Circle((int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']), int2inspect.iloc[i]['radius'], color='b', fill=False)
    plt.gcf().gca().add_artist(circle)
    plt.text(int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']+int2inspect.iloc[i]['radius'], 
             str(int2inspect.iloc[i]['id'])+', r='+str(int(int2inspect.iloc[i]['radius'])), 
             fontsize=11, color='b')
plt.axis('equal')
print(int2inspect['id'].sort_values().values)

In [ ]:
indices_list = {'d1':[1,5,13], 'd2':[6,14], 
                'd3':[10,23], 'd4':[0,2,3,4,9,18], 
                'd5':[10,12], 'd6':[0,6,24], 
                'd7':[5,6,10,12,14,16,23], 'd8':[0,1,4,6,7,16,17,19,35],
                'd9':[0,5,7,11,14,16,17,20,28,44], 'd10':[]}
indices = indices_list[dx]

shrink_factors_list = {'d1':[0.7,0.7,0.9], 'd2':[0.8,1.3], 
                       'd3':[0.85,0.8], 'd4':[0.85,0.8,0.8,0.8,0.9,0.9], 
                       'd5':[1.4,1.6], 'd6':[0.75,0.9,1.5], 
                       'd7':[0.7,0.9,1.5,0.6,0.4,1.6,0.8], 'd8':[0.8,0.75,0.85,0.9,0.8,0.7,0.9,0.9,0.7],
                       'd9':[0.8,0.85,0.7,0.7,1.5,1.3,1.1,0.9,0.75,2], 'd10':[]}
shrink_factors = shrink_factors_list[dx]

for idx, shrink_factor in zip(indices, shrink_factors):
    intersections.loc[(intersections['dx']==dx)&(intersections['id']==idx), 'radius'] *= shrink_factor

### Signalised or not

In [ ]:
cmap = mpl.colors.ListedColormap(['tab:blue', 'tab:orange'])

In [ ]:
int2inspect = intersections[(intersections['dx']==dx)&(intersections['keep'])].copy()
int2inspect[['x','y']] = (int2inspect[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
data = pd.read_hdf(os.path.abspath('..') + '/InputData/pNEUMA/'+dx+'/data_20181029_'+dx+'_0930_1000.h5', key='data').reset_index()
selected_frames = data['frame_id'].drop_duplicates().values
selected_frames = selected_frames[np.arange(5,len(selected_frames),15)]
data = data[data['frame_id'].isin(selected_frames)]
data.sort_values(['track_id', 'frame_id'])
data.loc[data['agent_type']!='Motorcycle', 'agent_type'] = 1
data.loc[data['agent_type']=='Motorcycle', 'agent_type'] = 2
data[['x','y']] = (data[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

check_id = -1

In [ ]:
check_id -= 2

In [ ]:
# repeat for each intersection
check_id += 1
intersection = int2inspect.iloc[check_id]
data_int = data[(data['x']-intersection['x'])**2+(data['y']-intersection['y'])**2<intersection['radius']**2]

for frame in data_int['frame_id'].unique():
    df = data_int[data_int['frame_id']==frame]
    fig, ax = plt.subplots(figsize=(4,4))
    ax.scatter(data_int['x'], data_int['y'], s=0.1, c='gainsboro')
    ax.scatter(df['x'], df['y'], c=df['agent_type'], cmap=cmap)
    for track in df['track_id'].unique():
        df_track = data_int[(data_int['track_id']==track)&(data_int['frame_id']<=frame)]
        color = 'tab:blue' if df_track['agent_type'].iloc[0]==1 else 'tab:orange'
        ax.plot(df_track['x'], df_track['y'], c=color, lw=0.5)
    ax.set_xlim([intersection['x']-intersection['radius'], intersection['x']+intersection['radius']])
    ax.set_ylim([intersection['y']-intersection['radius'], intersection['y']+intersection['radius']])
    ax.set_aspect('equal')
    ax.set_title('id: ' + str(intersection['id']) + ', time: ' + str(df['time'].iloc[0]))
    clear_output(wait=True)
    display(fig)
    # time.sleep(0.1)
    plt.close()

In [ ]:
dx

In [ ]:
unsignalized_list = {'d1':[2,6,13], 'd2':[18,], 
                     'd3':[], 'd4':[16], 
                     'd5':[12], 'd6':[], 
                     'd7':[10], 'd8':[0,2,19,20,27,36], 
                     'd9':[7,11,14,16,20,26,42,44], 'd10':[]}
unsignalized = unsignalized_list[dx]

# # In addition, remove intersections that have too few vehicles crossing through -- and cannot be determined as signalized or unsignalized
# remove_list = {'d1':[], 'd2':[], 
#                'd3':[], 'd4':[], 
#                'd5':[], 'd6':[], 
#                'd7':[10], 'd8':[17,4,20,10], 
#                'd9':[28,7,17], 'd10':[14]}
# remove = remove_list[dx]

intersections.loc[(intersections['dx']==dx)&(intersections['id'].isin(unsignalized)), 'signalized'] = 0
# intersections.loc[(intersections['dx']==dx)&(intersections['id'].isin(remove)), 'keep'] = False
# intersections.loc[(intersections['dx']==dx)&(intersections['id'].isin(remove)), 'signalized'] = -1

### Update information and save selected intersections

In [ ]:
intersections[(intersections['dx']==dx)]

In [ ]:
intersections.to_csv(data_path + 'intersections_all.csv', index=False)
intersections = intersections[intersections['keep']]
intersections.drop(columns=['keep']).to_csv(data_path + 'selected_intersections.csv', index=False)

## Merge intersections in overlapping areas

In [ ]:
# selected_intersections = pd.read_csv(data_path + 'selected_intersections.csv')
selected_intersections = pd.read_csv(data_path + 'target_intersections.csv')

roads = []
for dx in tqdm(['d'+str(did+1) for did in range(10)]):
    road = pd.read_hdf(data_path+'trajectories/'+dx+'_roads.h5', key='data')
    road['dx'] = dx
    roads.append(road)
roads = pd.concat(roads)

x0, y0 = trajectories[['x','y']].mean().values
roads[['x','y']] = (roads[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
int2plot = selected_intersections.copy()
int2plot[['x','y']] = (selected_intersections[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

In [ ]:
xlim = [0,2500]
road2plot = roads[(roads['x']>xlim[0])&(roads['x']<xlim[1])]
ints2inspect = int2plot[(selected_intersections['x']>xlim[0])&(selected_intersections['x']<xlim[1])]
fig, ax = plt.subplots(figsize=(15,15))
ax.scatter(road2plot['x'], road2plot['y'], s=0.1, c='grey')

int2inspect = ints2inspect[ints2inspect['signalized']>0.5]
for i in range(len(int2inspect)):
    ax.scatter(int2inspect['x'], int2inspect['y'], s=5, c='b')
    circle = plt.Circle((int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']), int2inspect.iloc[i]['radius'], color='b', fill=False)
    ax.add_artist(circle)
    # ax.text(int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']+int2inspect.iloc[i]['radius'], 
    #          str(int2inspect.iloc[i]['dx'])+'+'+str(int(int2inspect.iloc[i]['id'])), 
    #          fontsize=11, color='b')
    
int2inspect = ints2inspect[ints2inspect['signalized']<0.5]
for i in range(len(int2inspect)):
    ax.scatter(int2inspect['x'], int2inspect['y'], s=5, c='r')
    circle = plt.Circle((int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']), int2inspect.iloc[i]['radius'], color='r', fill=False)
    ax.add_artist(circle)
    # ax.text(int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']+int2inspect.iloc[i]['radius'], 
    #          str(int2inspect.iloc[i]['dx'])+'+'+str(int(int2inspect.iloc[i]['id'])), 
    #          fontsize=11, color='r')

ax.set_aspect('equal')

In [ ]:
selected_intersections['keep'] = True

remove_dx = ['d7','d9','d6','d9','d9','d9','d9','d6','d10','d6']
remove_id = [11,22,27,22,9,5,4,6,4,10]
selected_intersections.loc[(selected_intersections['dx'].isin(remove_dx))&(selected_intersections['id'].isin(remove_id)), 'keep'] = False

merge_points_i_dx = ['d7','d6','d9']
merge_points_i_id = [10,0,13]

merge_points_j_dx = ['d6','d9','d9']
merge_points_j_id = [24,0,28]

for i,j,dx_i,dx_j in zip(merge_points_i_id, merge_points_j_id, merge_points_i_dx, merge_points_j_dx):
    x_i, y_i, radius_i = selected_intersections.loc[(selected_intersections['dx']==dx_i)&(selected_intersections['id']==i), ['x','y','radius']].values[0]
    x_j, y_j, radius_j = selected_intersections.loc[(selected_intersections['dx']==dx_j)&(selected_intersections['id']==j), ['x','y','radius']].values[0]
    d = np.sqrt((x_i-x_j)**2+(y_i-y_j)**2)
    merged_radius = (radius_i + radius_j + d)/2
    merged_x = x_i + (x_j-x_i)*(merged_radius-radius_i)/d
    merged_y = y_i + (y_j-y_i)*(merged_radius-radius_i)/d
    selected_intersections.loc[(selected_intersections['dx']==dx_i)&(selected_intersections['id']==i), ['x','y','radius']] = [merged_x, merged_y, merged_radius]
    selected_intersections.loc[(selected_intersections['dx']==dx_j)&(selected_intersections['id']==j), 'keep'] = False
    selected_intersections.loc[(selected_intersections['dx']==dx_j)&(selected_intersections['id']==j), 'note'] = 'Merged with '+str(i)

selected_intersections.to_csv(data_path + 'selected_intersections.csv', index=False)

In [ ]:
selected_intersections = selected_intersections[selected_intersections['keep']].drop(columns=['keep','note'])
selected_intersections.to_csv(data_path + 'target_intersections.csv', index=False)

## Figure maker

Intersections

In [ ]:
figure_path = r'C:/SURFdrive/PhD progress/PhDResearch/1_DriverSpaceInference/Journal paper/Figures/'

In [ ]:
selected_intersections = pd.read_csv(data_path + 'target_intersections.csv')

In [ ]:
trajectories = []
roads = []
for dx in tqdm(['d'+str(did+1) for did in range(10)]):
    trajectory = pd.read_hdf(data_path+'trajectories/simplified_trajectories_'+dx+'.h5', key='data')
    trajectory['dx'] = dx
    road = pd.read_hdf(data_path+'trajectories/'+dx+'_roads.h5', key='data')
    road['dx'] = dx
    trajectories.append(trajectory)
    roads.append(road)
trajectories = pd.concat(trajectories)
roads = pd.concat(roads)

In [ ]:
traj2plot = trajectories.copy()
road2plot = roads.copy()
int2plot = selected_intersections.copy()

x0, y0 = trajectories[['x','y']].mean().values
traj2plot[['x','y']] = (trajectories[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
road2plot[['x','y']] = (roads[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
int2plot[['x','y']] = (selected_intersections[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))

lines = [line[['x','y']].values for _, line in road2plot.groupby(['dx','track_id'])[['x','y']]]
lc = mpl.collections.LineCollection(lines, color='k', linewidth=0.5, alpha=1, rasterized=True)
ax.add_collection(lc)
ax.set_xlim(road2plot['x'].min()-10, road2plot['x'].max()+10)
ax.set_ylim(road2plot['y'].min()-10, road2plot['y'].max()+10)
ax.set_aspect('equal')
ax.axis('off')

# Signalized intersections
int2inspect = int2plot[int2plot['signalized']>0.5]
for i in range(len(int2inspect)):
    circle = plt.Circle((int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']), int2inspect.iloc[i]['radius'], color='b', alpha=0.6, fill=False, ls='--', label='l')
    ax.add_artist(circle)

int2inspect = int2plot[int2plot['signalized']<0.5]
for i in range(len(int2inspect)):
    circle = plt.Circle((int2inspect.iloc[i]['x'], int2inspect.iloc[i]['y']), int2inspect.iloc[i]['radius'], color='r', fill=False)
    ax.add_artist(circle)

ax.set_aspect('equal')
c1 = plt.scatter([],[],label='Unsignalised intersection', marker='o', ec='r', fc='none', s=80)
c2 = plt.scatter([],[],label='Signalised intersection', marker='o', ec='b', fc='none', ls='--', alpha=0.6, s=80)
ax.legend(handles=[c1, c2], loc='upper right', fontsize=9, bbox_to_anchor=(1.0, 0.9), handletextpad=0.1)

In [ ]:
fig.savefig(figure_path+'Map_intersections.pdf', dpi=600, bbox_inches='tight')

In [ ]:
int2plot[int2plot['signalized']>0.5].shape

Straight roads

In [ ]:
traj2plot = trajectories.copy()
road2plot = roads.copy()
int2plot = selected_intersections.copy()

traj2plot[['x','y']] = (trajectories[['x','y']].values) @ rotateMatrix(-0.052*np.pi)
traj2plot[['hx','hy']] = (trajectories[['hx','hy']].values) @ rotateMatrix(-0.052*np.pi)
road2plot[['x','y']] = (roads[['x','y']].values) @ rotateMatrix(-0.052*np.pi)
int2plot[['x','y']] = (selected_intersections[['x','y']].values) @ rotateMatrix(-0.052*np.pi)

In [ ]:
trajectories['selected'] = 0
condition1 = ((traj2plot['x']>=385)&(traj2plot['x']<=425)&
              (traj2plot['y']>=1200)&(traj2plot['y']<=2100)&
              (abs(traj2plot['hx'])<=np.sin(10/180*np.pi))&(abs(traj2plot['hy'])>=np.cos(10/180*np.pi)))
condition2 = ((traj2plot['x']>=505)&(traj2plot['x']<=545)&
              (traj2plot['y']>=1200)&(traj2plot['y']<=2100)&
              (abs(traj2plot['hx'])<=np.sin(10/180*np.pi))&(abs(traj2plot['hy'])>=np.cos(10/180*np.pi)))
trajectories.loc[condition1, 'selected'] = 1
trajectories.loc[condition2, 'selected'] = 2

road1 = pd.DataFrame({'x':[425, 425, 385, 385], 'y':[1200, 2100, 1200, 2100]})
road2 = pd.DataFrame({'x':[545, 545, 505, 505], 'y':[1200, 2100, 1200, 2100]})
road1[['x','y']] = (road1[['x','y']].values) @ rotateMatrix(0.052*np.pi)
road2[['x','y']] = (road2[['x','y']].values) @ rotateMatrix(0.052*np.pi)

In [ ]:
traj2plot = trajectories.copy()
traj2plot[['x','y']] = (trajectories[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
road2plot[['x','y']] = (roads[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
int2plot[['x','y']] = (selected_intersections[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
road1[['x','y']] = (road1[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]
road2[['x','y']] = (road2[['x','y']].values-[x0,y0]) @ rotateMatrix(-0.305*np.pi) + [x0,y0]

In [ ]:
fig, ax = plt.subplots(figsize=(3.5,3))

lines = [line[['x','y']].values for _, line in road2plot.groupby(['dx','track_id'])[['x','y']]]
lc = mpl.collections.LineCollection(lines, color='k', linewidth=0.5, alpha=1)
ax.add_collection(lc)
points = MultiPoint(road1[['x','y']].values)
ax.plot(*points.convex_hull.exterior.xy, color='r', linewidth=1, alpha=1)
points = MultiPoint(road2[['x','y']].values)
ax.plot(*points.convex_hull.exterior.xy, color='r', linewidth=1, alpha=1)
ax.set_xlim(road2plot['x'].min()-10, road2plot['x'].max()+10)
ax.set_ylim(road2plot['y'].min()-10, road2plot['y'].max()+10)
ax.set_aspect('equal')
ax.axis('off')
legend = mpl.patches.Patch(edgecolor='r', facecolor='none', label='Selected roads')
ax.legend(handles=[legend], loc='upper right', fontsize=9, bbox_to_anchor=(0.95, 0.9))

In [ ]:
fig.savefig(figure_path+'Map_straight_roads.pdf', dpi=600, bbox_inches='tight')